In [ ]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar"]
launcher.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
import org.apache.http._
import org.apache.http.client.methods._
import org.apache.http.entity._
import org.apache.http.impl.client._
import org.apache.http.util._
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.json4s.jackson.Serialization

class SDK(url: String, pat: String) {
    val httpCookieStore = new BasicCookieStore()
    val builder = HttpClientBuilder.create().setDefaultCookieStore(httpCookieStore)
    implicit val formats = org.json4s.DefaultFormats        

    // cookie PAT
    val httpClient = builder.build();
    val request = new HttpPost(url+"/api/auth/callback?client_name=ExternalAppTokenClient");
    request.addHeader("X-External-App-Token", pat)
    httpClient.execute(request)
    httpClient.close

    def getJson(response: HttpResponse) = {
        val entity = response.getEntity
        val json = EntityUtils.toString(entity, java.nio.charset.StandardCharsets.UTF_8)
        parse(json).asInstanceOf[org.json4s.JObject]
    }
    
    def send(method:String, path: String, payload:Option[Map[String, Any]] = None, json: Boolean = true) = {
        val httpClient = builder.build()
        val urlPath = s"$url$path"
        val r = try {
            val request = if (method == "GET") {
                new HttpGet(urlPath)
            } else if (method == "DELETE") {
                new HttpDelete(urlPath)
            } else {
                val r = method match {
                    case "POST" => new HttpPost(urlPath)
                    case "PUT" => new HttpPut(urlPath)
                }
                payload.foreach { p =>
                    r.setHeader("Content-type", "application/json")
                    val stringEntity = new StringEntity(Serialization.write(p));
                    r.setEntity(stringEntity)
                }
                r
            }
            val response = httpClient.execute(request)
            if (json) getJson(response) else org.json4s.JObject()
        } finally { httpClient.close }
        r 
    }
    
    def getLineagesInProject(project: String, process: String, env: String, cv: String) = 
        send("GET", s"/business/api/views/v1/project-catalog/process/data-flow?projectId=$project&processId=$process&logical=true&environment=$env&codeVersionId=$cv")
    def getRules() = send("GET", "/business/api/views/v1/predicate-catalog")
    def getRulesForDs(ds_id: String, lineage_id: String, project_id: String, env: String) = 
        send("GET", s"/business/api/v1/performance/data/$ds_id/$lineage_id?projectId=$project_id&logical=true&environment=$env")
    def createRule(lds_id: String, lineage_id: String, project_id: String, process_id: String, env_name: String, field_name: String, fun: (String, Map[String, Any])) = 
        send("POST", "/business/api/v1/predicates", Some(Map(
                                                "context" -> "DATA_STATS",
                                                "datasourceId" -> lds_id,
                                                "lineageId" -> lineage_id,
                                                "projectId" -> project_id,
                                                "processId" -> process_id,
                                                "environment" -> env_name,
                                                "isLogical" -> true,
                                                "fieldName" -> field_name,
                                                "functionName" -> fun._1,
                                                "arguments" -> fun._2
                                            )))
    def updateRule(rule: String, fun: (String, Map[String, Any])) = 
        send("PUT", "/business/api/v1/predicates/" + rule, Some(Map("functionName" -> fun._1, "arguments" -> fun._2)), false)
    def deleteRule(rule: String) = send("DELETE", "/business/api/v1/predicates/"+rule)
}


In [ ]:
val pat = ""
val url = ""
val sdk = new SDK(url, pat)

In [ ]:
val allRules = sdk.getRules()

In [ ]:
def dsRules = (sdk.getRulesForDs(
    "logical-data-source-joined_data",
    "k-2a8a077a91f490348a95cbcb6d524e7c232ee4abe935cc68d25f01ea5980cdac",
    "k-ddc546028d6968a8b747d2b417d79c6d2ef650911422aa40a3283ea1492e4d57",
    "workshop"
) \ "data" \ "predicates").asInstanceOf[org.json4s.JArray]
dsRules

In [ ]:
val r = dsRules
if (r.values.size != 0) {
    // delete has issues...
    // sdk.deleteRule((r \\ "uuid").values.toString)
}

In [ ]:
dsRules

In [ ]:
sdk.createRule(
    "logical-data-source-joined_data",
    "k-2a8a077a91f490348a95cbcb6d524e7c232ee4abe935cc68d25f01ea5980cdac",
    "k-ddc546028d6968a8b747d2b417d79c6d2ef650911422aa40a3283ea1492e4d57",
    "k-ddb97a5623d591997c7e8b83caa660f2509958106614f2d3e5905d726f1a94d5",
    "workshop",
    "price.mean",
    ("Range", Map("minVal" -> 0, "maxVal" -> 1000))
)

In [ ]:
dsRules

In [ ]:
sdk.updateRule(
    (dsRules \\ "uuid").values.toString,
    ("Range", Map("minVal" -> 100, "maxVal" -> 1000))
)

In [ ]:
dsRules